##### Import Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import citipy
import requests
requests.__version__
from datetime import datetime
from config import weather_api_key
from scipy.stats import linregress

##### Create set of 2000 random latitudes and longitudes

In [11]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1990)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1990)
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list. -->unzipping
coordinates = list(lat_lngs)
coordinates

[(43.012682597484826, 129.07038262382434),
 (-84.89845409732203, 20.35125375067483),
 (85.67406143302526, 171.48834869902907),
 (-20.081827666664964, -8.433110005140378),
 (84.39481224196498, 158.8171899925551),
 (-3.4849224044122167, -74.61047266259324),
 (-56.6344385824914, -92.37211077952594),
 (13.690393125483567, -75.68795730700487),
 (19.32216354694691, 5.934303090963084),
 (-2.877550333852369, 162.10738253772678),
 (-12.649166217365561, 121.77380627408843),
 (-51.62502528153942, -60.76913198865448),
 (-74.28631440038293, 123.18224242056209),
 (-37.39998785473066, 159.05730627314773),
 (-21.777574654329726, -72.51851757063677),
 (-46.89029604192433, -31.586779912000196),
 (55.972180733244755, -118.11644702136059),
 (3.6542050698674586, -36.65139516721345),
 (-43.92143757739864, -86.6983076511413),
 (-52.335963180029964, -102.29738134367878),
 (-80.58771306479598, 136.43265612233665),
 (17.587041501596943, -140.0153647798281),
 (40.83037210368772, 155.2280262448068),
 (-66.7528622

##### Generate random world cities and find closest cities to lat_lngs coordinates

In [12]:
from citipy import citipy
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

753

##### Retrieve data from OpenWeatherMap

In [13]:
#Build URL for OpenWeatherMap
url= "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

city_url=f"{url}&q={city}"

In [14]:
#Get keys through json()
city_weather=requests.get(city_url)
city_weather=city_weather.json()

#Get all keys(in this case it's a list of dictionaries, each key is a dictionary)
# city_data.keys()
city_weather

{'coord': {'lon': -38.8828, 'lat': -15.8631},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 78.8,
  'feels_like': 82.89,
  'temp_min': 78.8,
  'temp_max': 78.8,
  'pressure': 1013,
  'humidity': 73},
 'visibility': 10000,
 'wind': {'speed': 5.75, 'deg': 40},
 'clouds': {'all': 0},
 'dt': 1616978360,
 'sys': {'type': 1,
  'id': 8422,
  'country': 'BR',
  'sunrise': 1616920830,
  'sunset': 1616964020},
 'timezone': -10800,
 'id': 3470137,
 'name': 'Belmonte',
 'cod': 200}

##### Get City Weather Data

In [15]:
#Create an empty list to hold the weather data
city_data=[]
#Print the beginning of the logging.
print("Beginning Data Retrieval")
print("------------------------")

#Create counters
record_count=1  #goes through each city in the list
set_count=1     #updates the sets once each set hits 50 cities


Beginning Data Retrieval
------------------------


In [16]:
#Loop through the list of cities and build the city URL

#for i in range(len(cities)): will on give us the index of the city in the list, to get both index and name we need 2 for loops
#instead of using 2 for loops, we use enumerate. 

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    #Run API requests for each of the cities:
    try:
        #Parse JSON and retrieve data:
        city_weather=requests.get(city_url).json()
        #Parse out the needed data:
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_desc = city_weather["weather"][0]["description"]

      
        #Append the city information (dictionary) into city_data list:
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_desc})

    #If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

#Indicate that Data Loading is complete.
print("--------------------------")
print("Data Retrieval Complete")
print("--------------------------")


Processing Record 1 of Set 1 | mingyue
Processing Record 2 of Set 1 | bredasdorp
Processing Record 3 of Set 1 | pevek
Processing Record 4 of Set 1 | jamestown
Processing Record 5 of Set 1 | cherskiy
Processing Record 6 of Set 1 | iquitos
Processing Record 7 of Set 1 | punta arenas
Processing Record 8 of Set 1 | puerto colombia
Processing Record 9 of Set 1 | arlit
Processing Record 10 of Set 1 | buala
Processing Record 11 of Set 1 | kupang
Processing Record 12 of Set 1 | ushuaia
Processing Record 13 of Set 1 | new norfolk
Processing Record 14 of Set 1 | nelson bay
Processing Record 15 of Set 1 | tocopilla
Processing Record 16 of Set 1 | cidreira
Processing Record 17 of Set 1 | fairview
Processing Record 18 of Set 1 | itarema
Processing Record 19 of Set 1 | castro
Processing Record 20 of Set 1 | hobart
Processing Record 21 of Set 1 | hilo
Processing Record 22 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 23 of Set 1 | east london
Processing Record 24 of Set 1 | sa

Processing Record 40 of Set 4 | ruatoria
City not found. Skipping...
Processing Record 41 of Set 4 | umm ruwabah
City not found. Skipping...
Processing Record 42 of Set 4 | ostrovnoy
Processing Record 43 of Set 4 | grindavik
Processing Record 44 of Set 4 | hay river
Processing Record 45 of Set 4 | puerto ayora
Processing Record 46 of Set 4 | karratha
Processing Record 47 of Set 4 | korla
Processing Record 48 of Set 4 | sars
Processing Record 49 of Set 4 | amderma
City not found. Skipping...
Processing Record 50 of Set 4 | traralgon
Processing Record 1 of Set 5 | gat
Processing Record 2 of Set 5 | miraflores
Processing Record 3 of Set 5 | sambava
Processing Record 4 of Set 5 | adrar
Processing Record 5 of Set 5 | erenhot
Processing Record 6 of Set 5 | bay roberts
Processing Record 7 of Set 5 | komatsu
Processing Record 8 of Set 5 | ukiah
Processing Record 9 of Set 5 | redmond
Processing Record 10 of Set 5 | tumannyy
City not found. Skipping...
Processing Record 11 of Set 5 | kendari
Pro

Processing Record 33 of Set 8 | aklavik
Processing Record 34 of Set 8 | vestmannaeyjar
Processing Record 35 of Set 8 | port-gentil
Processing Record 36 of Set 8 | sawtell
Processing Record 37 of Set 8 | quang ngai
Processing Record 38 of Set 8 | khuchni
Processing Record 39 of Set 8 | dawei
Processing Record 40 of Set 8 | salinas
Processing Record 41 of Set 8 | marzuq
Processing Record 42 of Set 8 | manokwari
Processing Record 43 of Set 8 | konyshevka
Processing Record 44 of Set 8 | pangnirtung
Processing Record 45 of Set 8 | gollere
City not found. Skipping...
Processing Record 46 of Set 8 | yithion
City not found. Skipping...
Processing Record 47 of Set 8 | steamboat springs
Processing Record 48 of Set 8 | pontian kecil
Processing Record 49 of Set 8 | oyama
Processing Record 50 of Set 8 | walvis bay
Processing Record 1 of Set 9 | bedele
Processing Record 2 of Set 9 | barabai
Processing Record 3 of Set 9 | namibe
Processing Record 4 of Set 9 | calvinia
Processing Record 5 of Set 9 | n

Processing Record 20 of Set 12 | porbandar
Processing Record 21 of Set 12 | upington
Processing Record 22 of Set 12 | takoradi
Processing Record 23 of Set 12 | nemyriv
Processing Record 24 of Set 12 | baruun-urt
Processing Record 25 of Set 12 | mattru
Processing Record 26 of Set 12 | verkhniy baskunchak
Processing Record 27 of Set 12 | boyolangu
Processing Record 28 of Set 12 | uxbridge
Processing Record 29 of Set 12 | piacabucu
Processing Record 30 of Set 12 | karkaralinsk
City not found. Skipping...
Processing Record 31 of Set 12 | noumea
Processing Record 32 of Set 12 | ambatondrazaka
Processing Record 33 of Set 12 | neya
Processing Record 34 of Set 12 | guerrero negro
Processing Record 35 of Set 12 | east wenatchee bench
Processing Record 36 of Set 12 | uribia
Processing Record 37 of Set 12 | sliven
Processing Record 38 of Set 12 | inta
Processing Record 39 of Set 12 | solovetskiy
City not found. Skipping...
Processing Record 40 of Set 12 | flin flon
Processing Record 41 of Set 12 

In [17]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Mingyue,43.1069,128.9217,41.38,61,49,15.01,CN,scattered clouds
1,Bredasdorp,-34.5322,20.0403,60.80,88,100,12.66,ZA,overcast clouds
2,Pevek,69.7008,170.3133,-8.37,94,78,5.91,RU,broken clouds
3,Jamestown,42.0970,-79.2353,37.99,93,90,22.97,US,mist
4,Cherskiy,68.7500,161.3000,1.02,84,99,4.97,RU,overcast clouds


In [18]:
new_column_order = ["City","Country","Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Mingyue,CN,43.1069,128.9217,41.38,61,49,15.01,scattered clouds
1,Bredasdorp,ZA,-34.5322,20.0403,60.80,88,100,12.66,overcast clouds
2,Pevek,RU,69.7008,170.3133,-8.37,94,78,5.91,broken clouds
3,Jamestown,US,42.0970,-79.2353,37.99,93,90,22.97,mist
4,Cherskiy,RU,68.7500,161.3000,1.02,84,99,4.97,overcast clouds


##### Output to csv

In [19]:
output_data_file = "WeatherPy_Database.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")